In [ ]:
import pandas as pd
import os
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file

In [ ]:
def from_bib_to_csv(diag_bib_raw):
    bib_data = []
    bib_columns = ['bibkey', 'type', 'title', 'authors', 'doi', 'gs_citations', 'journal', 'year', 'all_ss_ids', 'pmid']
    
    for bib_entry in diag_bib_raw:
        if bib_entry.type == 'string':
            continue

        bibkey = bib_entry.key
        bib_type = bib_entry.type
        fields = bib_entry.fields
        
        bib_authors = fields.get('author', '').strip('{}')
        bib_title = fields.get('title', '').strip('{}')
        bib_doi = fields.get('doi', '').strip('{}')
        bib_gscites = fields.get('gscites', '').strip('{}')
        bib_journal = fields.get('journal', '').strip('{}')
        bib_year = fields.get('year', '').strip('{}')
        bib_all_ss_ids = fields.get('all_ss_ids', '').strip('{}')
        bib_pmid = fields.get('pmid', '').strip('{}')
        
        bib_data.append([bibkey, bib_type, bib_title, bib_authors, bib_doi, bib_gscites, bib_journal, bib_year, bib_all_ss_ids, bib_pmid])

    df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)
    return df_bib_data

In [ ]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib)
bibfile = from_bib_to_csv(diag_bib_raw)
path_new_bib = os.path.join('script_data/', 'diag_ss_new.bib')
updated_bib_raw = read_bibfile(None, path_new_bib)
newbibfile = from_bib_to_csv(updated_bib_raw)

In [ ]:
new_bibkeys = newbibfile['bibkey'].tolist()

In [ ]:
old_bibkeys = bibfile['bibkey'].tolist()

In [ ]:
def get_entry(bibfile, bibkey_toupdate):
    for entry in bibfile:
        bibkey = entry.key
        if bibkey == bibkey_toupdate:
            return entry

In [ ]:
# Update existing bibitems
for entry in diag_bib_raw:
    if entry.type == 'string':
        continue
    bibkey = entry.key
    if bibkey in new_bibkeys:
        corresponding_entry = get_entry(updated_bib_raw, bibkey)
        if 'all_ss_ids' in corresponding_entry.fields:
            entry.fields['all_ss_ids'] = corresponding_entry.fields['all_ss_ids']
        if 'pmid' in corresponding_entry.fields:
            entry.fields['pmid'] = corresponding_entry.fields['pmid']

In [ ]:
bibkeys_toadd = set(new_bibkeys)-set(old_bibkeys)
df_to_add = newbibfile[newbibfile['bibkey'].isin(bibkeys_toadd)]

In [ ]:
bibkeys_toadd

In [ ]:
for entry in updated_bib_raw:
    bibkey = entry.key
    if bibkey in bibkeys_toadd:
        diag_bib_raw.append(entry)

In [ ]:
csv=from_bib_to_csv(diag_bib_raw)

In [ ]:
path_diag_bib = os.path.join('..', 'diag_latest_try.bib')
save_to_file(diag_bib_raw, path_diag_bib)